# Homework 2 Solutions

BEE 4850/5850, Fall 2024

**Name**:

**ID**:

> **Due Date**
>
> Friday, 2/23/24, 9:00pm

## Overview

### Instructions

The goal of this homework assignment is to practice developing and
working with probability models for data.

-   Problem 1 asks you to fit a sea-level rise model using normal
    residuals and to assess the validity of that assumption.
-   Problem 2 asks you to model the time series of hourly
    weather-related variability at a tide gauge.
-   Problem 3 asks you to model the occurrences of Cayuga Lake freezing,
    and is only slightly adapted from Example 4.1 in [Statistical
    Methods in the Atmospheric
    Sciences](https://www.sciencedirect.com/book/9780128158234/statistical-methods-in-the-atmospheric-sciences)
    by Daniel Wilks.
-   Problem 4 (**graded only for graduate students**) asks you to
    revisit the sea-level model in Problem 1 by including a model-data
    discrepancy term in the model calibration.

### Learning Outcomes

After completing this assignments, students will be able to:

-   develop probability models for data and model residuals under a
    variety of statistical assumptions;
-   evaluate the appropriateness of those assumptions through the use of
    qualitative and quantitative evaluations of goodness-of-fit;
-   fit a basic Bayesian model to data.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

The following packages are included in the environment (to help you find
other similar packages in other languages). The code below loads these
packages for use in the subsequent notebook (the desired functionality
for each package is commented next to the package).

In [2]:
using Random # random number generation and seed-setting
using DataFrames # tabular data structure
using CSVFiles # reads/writes .csv files
using Distributions # interface to work with probability distributions
using Plots # plotting library
using StatsBase # statistical quantities like mean, median, etc
using StatsPlots # some additional statistical plotting tools
using Optim # optimization tools

Random.seed!(1)

## Problems (Total: 30 Points for 4850; 40 for 5850)

### Problem 1

Consider the following sea-level rise model from [Rahmstorf
(2007)](https://doi.org/10.1073/pnas.0907765106):

$$\frac{dH(t)}{dt} = \alpha (T(t) - T_0),$$ where $T_0$ is the
temperature (in $^\circ C$) where sea-level is in equilibrium
($dH/dt = 0$), and $\alpha$ is the sea-level rise sensitivity to
temperature. Discretizing this equation using the Euler method and using
an annual timestep ($\delta t = 1$), we get
$$H(t+1) = H(t) + \alpha (T(t) - T_0).$$

**In this problem**:

-   Load the data from the `data/` folder
    -   Global mean temperature data from the HadCRUT 5.0.2.0 dataset
        (<https://hadobs.metoffice.gov.uk/hadcrut5/data/HadCRUT.5.0.2.0/download.html>)
        can be found in
        `data/HadCRUT.5.0.2.0.analysis.summary_series.global.annual.csv`.
        This data is averaged over the Northern and Southern Hemispheres
        and over the whole year.
    -   Global mean sea level anomalies (relative to the 1990 mean
        global sea level) are in `data/CSIRO_Recons_gmsl_yr_2015.csv`,
        courtesy of CSIRO
        (<https://www.cmar.csiro.au/sealevel/sl_data_cmar.html>).
-   Fit the model under the assumption of normal i.i.d. residuals by
    maximizing the likelihood and report the parameter estimates. Note
    that you will need another parameter $H_0$ for the initial sea
    level. What can you conclude about the relationship between global
    mean temperature increases and global mean sea level rise?
-   How appropriate was the normal i.i.d. probability model for the
    residuals? Use any needed quantitative or qualitative assessments of
    goodness of fit to justify your answer. If this was not an
    appropriate probability model, what would you change?

***Solution***:

First, let’s load the data and implement the model with a function.

In [3]:
# load data files
slr_data = DataFrame(load("data/CSIRO_Recons_gmsl_yr_2015.csv"))
gmt_data = DataFrame(load("data/HadCRUT.5.0.2.0.analysis.summary_series.global.annual.csv"))
slr_data[:, :Time] = slr_data[:, :Time] .- 0.5; # remove 0.5 from Times
dat = leftjoin(slr_data, gmt_data, on="Time") # join data frames on time
select!(dat, [1, 2, 3, 4])  # drop columns we don't need
first(dat, 6)

In [4]:
# slr_model: function to simulate sea-level rise from global mean temperature based on the Rahmstorf (2007) model

function slr_model(α, T₀, H₀, temp_data)
    temp_effect = α .* (temp_data .- T₀)
    slr_predict = cumsum(temp_effect) .+ H₀
    return slr_predict
end

slr_model (generic function with 1 method)

Now, let’s fit the model under the normal residual assumption.

In [5]:
# split data structure into individual pieces
years = dat[:, 1]
sealevels = dat[:, 2]
temp = dat[:, 4]

# write function to calculate likelihood of residuals for given parameters
# parameters are a vector [α, T₀, H₀, σ]
function llik_normal(params, temp_data, slr_data)
    slr_out = slr_model(params[1], params[2], params[3], temp_data)
    resids = slr_out - slr_data
    return sum(logpdf.(Normal(0, params[4]), resids))
end

# set up lower and upper bounds for the parameters for the optimization
lbds = [0.0, -50.0, -200.0, 0.0]
ubds = [10.0, 1.0, 0.0, 20.0]
p0 = [5.0, -1.0, -100.0, 5.0]
p_mle = Optim.optimize(p -> -llik_normal(p, temp, sealevels), lbds, ubds, p0).minimizer

4-element Vector{Float64}:
    1.863932768247625
   -0.9709731494908135
 -157.3346053534705
    5.911511282104595

As `p_mle[1]` has a positive value of 1.86, this suggests a positive
correlation between global mean temperature and global mean sea levels.
Of course, we can’t conclude anything about causality, as it could be
that both are independently increasing simultaneously, or that there is
a mutual cause of both increases. However, mechanistically it makes
sense that temperature increases would increase sea levels, due to
melting ice and thermal expansion of ocean waters.

To look at the appropriateness of the normal residual assumption, let’s
compute the residuals and look at a Q-Q plot and their autocorrelation.

In [6]:
# compute the residuals
resids = slr_model(p_mle[1], p_mle[2], p_mle[3], temp) - sealevels
histogram(resids, xlabel="Residual", ylabel="Count", legend=false)

The residual histogram in
<a href="#fig-p1-histogram" class="quarto-xref">Figure 1</a> looks a bit
skewed. Let’s look at a Q-Q plot.

In [7]:
qqplot(Normal, resids, xlabel="Normal Theoretical Quantile", ylabel="Sample Residual Quantile", legend=false)

The Q-Q plot (<a href="#fig-p1-qq" class="quarto-xref">Figure 2</a>)
looks good; there are some discrepancies near the tail, but that is not
surprising.

In [8]:
resid_acf = pacf(resids, 1:5)
plot(resid_acf , marker=:circle, line=:stem, linewidth=3, markersize=5, legend=false, ylabel="Partial Autocorrelation", xlabel="Time Lag")

The bigger problem is that the lag-1 autocorrelation coefficient is
0.54, which is not consistent with the normally-distributed assumption.
The next thing to try would be an AR(1) model for the residuals.

### Problem 2

Tide gauge data is complicated to analyze because it is influenced by
different harmonic processes (such as the linear cycle). In this
problem, we will develop a model for this data using [NOAA data from the
Sewell’s Point tide
gauge](https://tidesandcurrents.noaa.gov/waterlevels.html?id=8638610)
outside of Norfolk, VA from `data/norfolk-hourly-surge-2015.csv`. This
is hourly data (in m) from 2015 and includes both the observed data
(`Verified (m)`) and the tide level predicted by NOAA’s sinusoidal model
for periodic variability, such as tides and other seasonal cycles
(`Predicted (m)`).

**In this problem**:

-   Load the data file. Take the difference between the observations and
    the sinusoidal predictions to obtain the tide level which could be
    attributed to weather-related variability (since for one year
    sea-level rise and other factors are unlikely to matter). Plot this
    data.
-   Develop an autoregressive model for the weather-related variability
    in the Norfolk tide gauge. Make sure to include your logic or
    exploratory analysis used in determining the model specification.
-   Use your model to simulate 1,000 realizations of hourly tide gauge
    observations. What is the distribution of the maximum tide level?
    How does this compare to the observed value?

***Solution***:

First, load the data, find the weather-related residuals, and plot. To
load and process the data, we will use the `DataFramesMeta.jl` package,
which lets us string together commands in a convenient way. This is
completely optional, however.

In [9]:
tide_dat = DataFrame(load("data/norfolk-hourly-surge-2015.csv"))
surge_resids = tide_dat[:, 5] - tide_dat[:, 3]
plot(surge_resids; ylabel="Gauge Measurement (m)", label="Weather-Related Residual", legend=:topleft, xlabel="Hour Number")

Let’s look at the autocorrelation function of the residuals to determine
what the autoregressive lag should be.

In [10]:
resid_acf = pacf(surge_resids, 1:5)
plot(resid_acf , marker=:circle, line=:stem, linewidth=3, markersize=5, legend=false, ylabel="Partial Autocorrelation", xlabel="Time Lag")

We can see that there is a *very* strong autocorrelation at lag 1, but
not much additional autocorrelation introduced at lag 2, so we will use
an AR(1) model (though you might be justified in using an AR(2) model,
as the lag-2 partial autocorrelation is not completely negligible).

This AR(1) model is
$$y_t = \rho y_{t+1} + \varepsilon, \quad \varepsilon \sim N(0, \sigma).$$
Let’s implement this model and fit it. We’ll specify the likelihood
based on whitening the residuals and computing the individual normal
likelihoods of the resulting terms.

In [11]:
function llik_ar(ρ, σ, y)
    ll = 0 # initialize log-likelihood counter
    ll += logpdf(Normal(0, σ/sqrt(1-ρ^2)), y[1])
    for i = 1:length(y)-1
        residuals_whitened = y[i+1] - ρ * y[i]
        ll += logpdf(Normal(0, σ), residuals_whitened)
    end
    return ll
end

# set up lower and upper bounds for the parameters for the optimization
lbds = [0.0, 0.0]
ubds = [1.0, 5.0]
p0 = [0.5, 2.0]
p_tide_mle = Optim.optimize(p -> -llik_ar(p[1], p[2], surge_resids), lbds, ubds, p0).minimizer

2-element Vector{Float64}:
 0.9956392098280269
 0.019663589611082367

Now we can simulate these residuals and add them back to the NOAA model
predictions using the `mapslices()` function. The resulting histogram of
the maximum values is shown in
<a href="#fig-p2-histogram" class="quarto-xref">Figure 6</a>.

In [12]:
# T is the length of the simulated series, N the number of simulations
function simulate_ar1(ρ, σ, T, N)
    y = zeros(T, N) # initialize storage
    y[1, :] = rand(Normal(0, σ/sqrt(1-ρ^2)), N) # sample initial value
    # simulate the remaining residuals
    for t = 2:T
        y[t, :] = ρ * y[t-1, :] + rand(Normal(0, σ), N)
    end
    return y
end

# simulate new observations
simulated_resids = simulate_ar1(p_tide_mle[1], p_tide_mle[2], length(surge_resids), 1000)
simulated_obs = mapslices(col -> col + tide_dat[:, 3], simulated_resids, dims=1)
# plot histogram
max_obs = [maximum(simulated_obs[:, n]) for n in 1:size(simulated_obs, 2)]
histogram(max_obs, xlabel="Maximum Simulated Gauge Level (m)", ylabel="Count", legend=false)

This distribution fails to capture the observed maximum 1.98, which
could be due to a model structural problem (we might want to allow for
more extreme AR(1) innovations than the normal distribution of
$\varepsilon_t$) or just the unlikeliness of that one observation.

### Problem 3

As of 2010, Cayuga Lake has frozen in the following years: 1796, 1816,
1856, 1875, 1884, 1904, 1912, 1934, 1961, and 1979. Based on this data,
we would like to project whether Cayuga Lake is likely to freeze again
in the next 25 years.

**In this problem**:

-   Assuming that observations began in 1780, write down a Bayesian
    model for whether Cayuga Lake will freeze in a given year, using a
    Bernoulli distribution. How did you select what prior to use?
-   Find the maximum *a posteriori* estimate using your model.
-   Generate 1,000 realizations of Cayuga Lake freezing occurrences from
    1780 to 2010 and check the simulations against the occurrance data.
-   Using your model, calculate the probability of Cayuga Lake freezing
    at least once in the next 10 years.
-   What do you think about the validity of your model, both in terms of
    its ability to reproduce historical data and its use to make future
    projections? Why might you believe or discount it? What changes
    might you make (include thoughts about the prior)?

***Solution***:

First, let’s enter the data.

In [13]:
fz_yrs = [1796, 1816, 1856, 1875, 1884, 1904, 1912, 1934, 1961, 1979]
y = DataFrame(year=1780:2010, freeze=zeros(Bool, length(1780:2010)))
# indexin(a, b) finds indices of a which are in b and returns nothing if not found
# so we use !isnothing to only get the indices which occur in fz_yrs, and (.!) is needed
# to broadcast
y[.!(isnothing.(indexin(y.year, fz_yrs))), :freeze] .= true

10-element view(::Vector{Bool}, [17, 37, 77, 96, 105, 125, 133, 155, 182, 200]) with eltype Bool:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

Using a Bernoulli distribution means that we need
$y_t \sim Bernoulli(p)$, where $p$ is the probability of freezing in
year $t$. The maximum-likelihood estimate of $p$ is easy: it’s just the
empirical frequency of `true`. But we want to develop a Bayesian model,
as this empirically-observed frequency might not actually be the most
likely outcome. To do this, we need to specify a prior for $p$, which is
usually given as $p \sim Beta(\alpha, \beta)$. We will use prior
predictive simulations to determine appropriate Beta parameters.

In [14]:
function sim_freeze(α, β, nyrs, nsamples)
    p = rand(Beta(α, β), nsamples)
    fz = sum.(rand.(Bernoulli.(p), nyrs))
    return fz
end

sim_freeze (generic function with 1 method)

Let’s see what the impact of a $Beta(1, 10)$ prior and a $Beta(1, 20)$
prior look like.

In [15]:
fz_10 = sim_freeze(1, 10, nrow(y), 1000)
p1 = histogram(fz_10, xlabel="Number of Freezes", ylabel="Count", legend=false)
plot!(p1, size=(300, 250))

fz_20 = sim_freeze(1, 20, nrow(y), 1000)
p2 = histogram(fz_20, xlabel="Number of Freezes", ylabel="Count", legend=false)
plot!(p2, size=(300, 250))

display(p1)
display(p2)

<a href="#fig-freeze-prior-2" class="quarto-xref">Figure 7 (b)</a> looks
reasonable, since we might suspect that freezing shouldn’t occur too
frequently. As a result, we’ll work with the $p \sim Beta(1, 20)$ prior
for this model. So the full model specification is:

Now we can compute the MAP estimate.

In [16]:
# compute the log-posterior
function lpost(p, dat)
    lpri = logpdf(Beta(1, 20), p)
    llik = sum(logpdf.(Bernoulli(p), dat))
    return lpri + llik
end

map = Optim.optimize(p -> -lpost(p, y[:, :freeze]), 0, 1).minimizer

0.039999999097374374

That gives us a MAP estimate of 0.039999999097374374. Note that your
answer might differ depending on the prior you selected. Now we can do
simulations and compare the results to the observations. We will do this
using histograms to show the number of occurrences, as the model does
not account for any temporal changes in frequency.

In [17]:
function sim_freeze_map(p, nyrs, nsamples)
    fz = mapslices(sum, rand(Bernoulli(p), (nyrs, nsamples)), dims=1)'
    return fz
end

p = histogram(sim_freeze_map(map, nrow(y), 1000), xlabel="Number of Freezing Occurrences", ylabel="Count", label=false)
vline!(p, [sum(y[:, :freeze])], label="Data")

The probability of Cayuga Lake freezing in the next 10 years is
calculated with:

In [18]:
100 * sum(sim_freeze_map(map, 10, 1000)) / (10*1000)

4.22

Based on <a href="#fig-freeze-hist" class="quarto-xref">Figure 8</a>,
the model seems to replicate the data well (with a slight bias towards
fewer freezing events due to the prior). However, the model might be
improved with time or temperature dependence, as the original data
appears to have more freezing events earlier in the record and fewer
after 1950.

### Problem 4

<span style="color:red;">GRADED FOR 5850 STUDENTS ONLY</span>

For the sea-level model in Problem 1, model the model-data discrepancy
using an AR(1) process, with observation error modeled as normally
distributed with standard deviation given by the uncertainty column in
the data file.

**In this problem**:

-   Find the maximum likelihood estimate of the parameters with this
    discrepancy structure. How does the parameter inference change from
    the normal i.i.d. estimate in Problem 1?
-   Generate 1,000 traces, plot a comparison of the hindcasts to those
    from Problem 1, and compare the surprise indices.
-   Determine whether you have accounted for autocorrelation in the
    residuals appropriately (hint: generate realizations of just the
    discrepancy series, compute the resulting residuals from the model
    fit + discrepancy, and look at the distribution of autocorrelation
    values).
-   Which model specification would you prefer and why?

:::

From the lecture, we can implement the discrepancy structure likelihood
as:

In [19]:
function ar_covariance_mat(σ, ρ, y_err)
    H = abs.((1:length(y_err)) .- (1:(length(y_err)))') # compute the outer product to get the correlation lags
    ζ_var = σ^2 / (1-ρ^2)
    Σ = ρ.^H * ζ_var
    for i in 1:length(y_err)
        Σ[i, i] += y_err[i]^2
    end
    return Σ
end

function ar_discrep_log_likelihood(p, σ, ρ, y, m, y_err)
    y_pred = m(p)
    residuals = y_pred .- y
    Σ = ar_covariance_mat(σ, ρ, y_err)
    ll = logpdf(MvNormal(zeros(length(y)), Σ), residuals)
    return ll
end

# params vector is [α, T₀, H₀]
slr_wrap(params) = slr_model(params[1], params[2], params[3], temp)

# maximize log-likelihood within some range
# important to make everything a Float instead of an Int 
lower = [0.0, -5.0, -200.0, 0.0, -1.0]
upper = [5.0, 1.0, 0.0, 20.0, 1.0]
p0 = [2.0, 0.0, -100.0, 5.0, 0.5]
result = Optim.optimize(params -> -ar_discrep_log_likelihood(params[1:end-2], params[end-1], params[end], sealevels, slr_wrap, dat[:, 3]), lower, upper, p0)
p_ar_mle = result.minimizer

5-element Vector{Float64}:
    1.998198372838903
   -0.9076726257161041
 -154.93253804174117
    1.3977277644676105
    0.8557998637644137

The AR discrepancy model has a higher sea-level sensitivity to
temperature and a slightly higher equilibrium temperature and initial
condition. Simulating a hindcast:

In [20]:
nsamples = 1000
Σ = ar_covariance_mat(p_ar_mle[4], p_ar_mle[5], dat[:, 3])
residuals_discrep = rand(MvNormal(zeros(nrow(dat)), Σ), nsamples)
model_discrep = slr_wrap(p_ar_mle[1:end-2])
model_sim_discrep = (residuals_discrep .+ model_discrep)'
median_discrep = mapslices(col -> quantile(col, 0.5), model_sim_discrep; dims=1)' # compute median
q90_discrep = mapslices(col -> quantile(col, [0.05, 0.95]), model_sim_discrep; dims=1) # compute 90% prediction interval
plot(years, median_discrep, linewidth=2, label="Discrepancy Model", ribbon=(median_discrep .- q90_discrep[1, :], q90_discrep[2, :] .- median_discrep), fillalpha=0.2, xlabel="Year", ylabel="Sea Level Anomaly (mm)")
scatter!(years, sealevels, label="Observations")

Calculating the surprise index
($1 - \frac{1}{n} \sum_{i=1}^n \mathbb{I}_{\mathcal{I}_\alpha}(y_i)$) of
both 90% intervals:

In [21]:
# need to compute the hindcast projection intervals for the iid model
slr_iid_out = slr_model(p_mle[1], p_mle[2], p_mle[3], temp)
iid_residuals = rand(Normal(0, p_mle[4]), (length(years), nsamples))
model_sim_iid = (iid_residuals .+ slr_iid_out)'
q90_iid = mapslices(col -> quantile(col, [0.05, 0.95]), model_sim_iid; dims=1) 

2×134 Matrix{Float64}:
 -165.159  -164.663  -161.948  …  46.7122  48.602   52.3031  55.1233
 -146.702  -145.918  -144.36      65.5852  69.0316  70.9158  74.4833

In [22]:
surprise_discrep = 0 # initialize surprise counter
surprise_iid = 0
# go through the data and check which points are outside of the 90% interval
for i = 1:length(sealevels)
    ## The || operator is an OR, so returns true if either of the terms are true
    if (sealevels[i] < q90_discrep[1, i]) || (q90_discrep[2, i] < sealevels[i])
        surprise_discrep += 1
    end
    if (sealevels[i] < q90_iid[1, i]) || (q90_iid[2, i] < sealevels[i])
        surprise_iid += 1
    end

end
si_iid = surprise_iid / length(sealevels) * 100
si_discrep = surprise_discrep / length(sealevels) * 100
[si_iid; si_discrep]

2-element Vector{Float64}:
 8.955223880597014
 1.4925373134328357

The AR(1) discrepancy model here actually looks underconfident, which
might suggest that the variance for the discrepancy process is too large
due to the large historical SLR observation errors (which could be
accounted for with a tighter prior in a Bayesian setup). But we should
also check if we did account for the residual autocorrelation that we
saw in Problem 1, which we can do by generating simulations without
observation error.

In [23]:
nsamples = 1000
Σ = ar_covariance_mat(p_ar_mle[4], p_ar_mle[5], zeros(nrow(dat)))
residuals_discrep = rand(MvNormal(zeros(nrow(dat)), Σ), nsamples)
model_discrep = slr_wrap(p_ar_mle[1:end-2])
model_sim_discrep = (residuals_discrep .+ model_discrep)'
obs_error = model_sim_discrep .- sealevels'
obs_error_acf = mapslices(col -> pacf(col, 1:5), obs_error, dims=2)
boxplot(obs_error_acf, legend=false, xlabel="Autocorrelation Lag", ylabel="Partial Autocorrelation")

<a href="#fig-sealevel-ar-acf" class="quarto-xref">Figure 10</a>
suggests that the lag-1 autocorrelation has not actually been accounted
for properly, which might suggest that there is a problem with the model
structure or that a more sophisticated error model is needed (for
example, linking the residuals to the global mean temperature or
accounting for clustering in the residual variance). Between the two
models, since neither model properly accounted for the lag-1 residual
autocorrelation, we might prefer the simpler i.i.d. normal model which
had a more appropriate surprise index.